In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=cf6d26a9a39db1fba7bd6a0aecce0f894d94e2eee2b0298cddc9476aa3ea0376
  Stored in directory: /root/.cache/pip/wheels/0c/c2/0e/3b9c6845c6a4e35beb90910cc70d9ac9ab5d47402bd62af0df
Successfully built ffmpy
  Attempting uninstall: mdit-py-plugins
    Found existing installation: mdit-py-plugins 0.3.5
    Uninstalling mdit-py-plugins-0.3.5:
      Successfully uninstalled mdit-py-plugins-0.3.5


In [2]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

model = AutoModelForTokenClassification.from_pretrained("patidarsourabh3/final_submisssion")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
def token_classification(input_text, flag=False):
    if flag:
        print("\033[1;31;47mProcess\033[1;32;47m Task\033[1;34;47m Material")
        print()
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
    predictions = outputs.logits.argmax(dim=-1)[0].tolist()
    unique_tags = ['O','B-Task', 'I-Task', 'B-Process', 'I-Process', 'B-Material', 'I-Material']
    ans = [unique_tags[i] for i in predictions]
    text = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    final_output = ""
    for i in range(len(text)):
        if flag:
            if ans[i]=='O':
                print("\033[0;30;47m"+text[i]+"\033[0;37;47m", end =" ")
            elif ans[i]=='B-Process' or ans[i]=='I-Process':
                print("\033[1;31;47m"+text[i]+"\033[0;37;47m",end=" ")
            elif ans[i]=='B-Task' or ans[i]=='I-Task':
                print("\033[1;32;47m"+text[i]+"\033[0;37;47m",end=" ")
            else:
                print("\033[1;34;47m"+text[i]+"\033[0;37;47m",end=" ")
        else:
            final_output += text[i]+"\t"+ans[i] + "\n"
    
    if flag==False:
        return final_output

In [16]:
# text = input()
text = '''
The study outlines a trial of transient response analysis on full-scale motorway bridge structures to obtain information concerning the steel–concrete interface and is part of a larger study to assess the long-term sustained benefits offered by Impressed Current Cathodic Protection (ICCP) after the interruption of the protective current [1]. These structures had previously been protected for 5–16years by an ICCP system prior to the start of the study. The protective current was interrupted, in order to assess the long-term benefits provided by ICCP after it has been turned off. This paper develops and examines a simplified approach for the on-site use of transient response analysis and discusses the potential advantages of the technique as a tool for the assessment of the corrosion condition of steel in reinforced concrete structures.
'''
token_classification(text, True)

Process Task Material

[CLS] the study outlines a trial of transient response analysis on full - scale motor ##way bridge structures to obtain information concerning the steel – concrete interface and is part of a larger study to assess the long - term sustained benefits offered by imp ##ressed current cath ##odic protection ( icc ##p ) after the interruption of the protective current [ 1 ] . these structures had previously been protected for 5 – 16 ##years by an icc ##p system prior to the start of the study . the protective current was interrupted , in order to assess the long - term benefits provided by icc ##p after it has been turned off . this paper develops and examines a simplified approach for the on - site use of transient response analysis and discusses the potential advantages of the technique as a tool for the assessment of the corrosion condition of steel in reinforced concrete structures . [SEP] 

In [17]:
import gradio as gr

demo = gr.Interface(
    fn=token_classification,
    inputs=gr.Textbox(lines=15, placeholder="Put input text here.......", label = "Input Text"),
    outputs=gr.Textbox(placeholder="Output will be seen here.......", label = "Output"),
)
demo.launch()

Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://140ae8b4f7c341ba51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
